# Modelo ER

![Modelo Er](https://raw.githubusercontent.com/JoelFrancisco/trabalho-pesquisa-arquitetura-de-dados/main/github/modelo_er.png)

In [1]:
WORK_DIR = "/home/jovyan/"

In [2]:
import pyspark
from delta import *

builder = pyspark.sql.SparkSession.builder.appName("trabalho-pesquisa-arquitetura-de-dados") \
    .config("spark.sql.extensions","io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog","org.apache.spark.sql.delta.catalog.DeltaCatalog")

spark = configure_spark_with_delta_pip(builder).getOrCreate()

In [3]:
df = spark.read.option("header", True).csv("/home/jovyan/data/Indain_Food_Cuisine_Dataset.csv")

In [4]:
df.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|        name_of_Dish|           Diet_Type|         Course_name|  Discrption_of_Dish|        Cuisine_name|     Ratings_of_Dish|      Similar_Dishes| Ingredients_of_Dish|     Prepration_time|        Cooking_time|          Total_time|               Makes| Recipe_Instructions|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|Gujarati Lasaniya...|    Diet: Vegetarian|   Course: Side Dish|Lasaniya Batata R...|['Cuisine: Gujara...|                 4.9|['Chatpata Baby P...|['Baby Potatoes',...|      

In [5]:
spark.sql("""CREATE TABLE dishe_region (id_region INTEGER, nome STRING) USING delta LOCATION '/home/jovyan/data/delta/dishe_region'""")

DataFrame[]

In [6]:
spark.sql("""CREATE TABLE diet_type (id_diet INTEGER, description STRING) USING delta LOCATION '/home/jovyan/data/delta/diet_type'""")

DataFrame[]

In [7]:
spark.sql("""CREATE TABLE dishe_instructions (id_instructions INTEGER, id_dishes INTEGER, instructions_seq INTEGER, description STRING) USING delta LOCATION '/home/jovyan/data/delta/dishe_instructions'""")

DataFrame[]

In [8]:
spark.sql("""CREATE TABLE dishe_type (id_region INTEGER, name STRING) USING delta LOCATION '/home/jovyan/data/delta/dishe_type'""")

DataFrame[]

In [9]:
spark.sql("""CREATE TABLE dishe_similiar (id_similiar_dishes INTEGER, id_dishes INTEGER, name STRING) USING delta LOCATION '/home/jovyan/data/delta/dishe_similiar'""")

DataFrame[]

In [10]:
spark.sql("""CREATE TABLE dishe_ingredients (id_ingredients INTEGER, id_dishes INTEGER, name STRING, description STRING, quantity FLOAT, unit_of_measurement STRING) USING delta LOCATION '/home/jovyan/data/delta/dishe_ingredients'""")

DataFrame[]

In [11]:
spark.sql("""CREATE TABLE dishes (id_dishes INTEGER, id_diet INTEGER, id_type INTEGER, description STRING, id_region INTEGER, rating INTEGER, id_similiar_dishes INTEGER, id_ingredients INTEGER, preparation_time INTEGER, cook_time INTEGER, makes INTEGER, id_instructions INTEGER) USING delta LOCATION '/home/jovyan/data/delta/dishes'""")

DataFrame[]

In [12]:
dishe_region = df.select('Cuisine_name')

In [ ]:
dishe_region.show()

In [14]:
from pyspark.sql.functions import monotonically_increasing_id

dishe_region = dishe_region.withColumn("id_region", monotonically_increasing_id().cast("int"))

In [15]:
dishe_region.show()

+--------------------+---------+
|        Cuisine_name|id_region|
+--------------------+---------+
|['Cuisine: Gujara...|        0|
|['Cuisine: North ...|        1|
|['Cuisine: Contin...|        2|
|['Cuisine: Mangal...|        3|
| ['Cuisine: Andhra']|        4|
|['Cuisine: Contin...|        5|
| 'Blend avocado a...|        6|
|['Cuisine: Coasta...|        7|
| ['Cuisine: Konkan']|        8|
|['Cuisine: Contin...|        9|
| 'You can serve i...|       10|
| in Bengali. Gene...|       11|
|['Cuisine: South ...|       12|
|['Cuisine: South ...|       13|
|['Cuisine: South ...|       14|
|['Cuisine: South ...|       15|
| ['Cuisine: Andhra']|       16|
|['Cuisine: Europe...|       17|
|['Cuisine: Contin...|       18|
| moist and delici...|       19|
+--------------------+---------+
only showing top 20 rows



In [16]:
dishe_region = dishe_region.withColumnRenamed("Cuisine_name", "nome")

In [17]:
dishe_region.write.format("delta").mode("append").save("/home/jovyan/data/delta/dishe_region")

In [18]:
result = spark.sql("SELECT * FROM dishe_region")
result.show()

+---------+--------------------+
|id_region|                nome|
+---------+--------------------+
|        0|['Cuisine: South ...|
|        1|['Cuisine: Uttar ...|
|        2| ['Cuisine: Indian']|
|        3|['Cuisine: Mangal...|
|        4|['Cuisine: Karnat...|
|        5|['Cuisine: South ...|
|        6| ['Cuisine: Indian']|
|        7|                60 M|
|        8|['Cuisine: North ...|
|        9|['Cuisine: North ...|
|       10| ['Cuisine: Indian']|
|       11|['Cuisine: Contin...|
|       12|['Cuisine: Contin...|
|       13| ['Cuisine: Indian']|
|       14| 'Let the peas be...|
|       15| ['Cuisine: Indian']|
|       16|['Cuisine: Kerala...|
|       17|['Cuisine: South ...|
|       18|  ['Cuisine: Asian']|
|       19| ['Cuisine: Indian']|
+---------+--------------------+
only showing top 20 rows



In [19]:
spark.sql("""DELETE FROM dishe_region WHERE id_region=1""")

DataFrame[num_affected_rows: bigint]

In [20]:
result = spark.sql("SELECT * FROM dishe_region")
result.show()

+---------+--------------------+
|id_region|                nome|
+---------+--------------------+
|        0|['Cuisine: Gujara...|
|        2|['Cuisine: Contin...|
|        3|['Cuisine: Mangal...|
|        4| ['Cuisine: Andhra']|
|        5|['Cuisine: Contin...|
|        6| 'Blend avocado a...|
|        7|['Cuisine: Coasta...|
|        8| ['Cuisine: Konkan']|
|        9|['Cuisine: Contin...|
|       10| 'You can serve i...|
|       11| in Bengali. Gene...|
|       12|['Cuisine: South ...|
|       13|['Cuisine: South ...|
|       14|['Cuisine: South ...|
|       15|['Cuisine: South ...|
|       16| ['Cuisine: Andhra']|
|       17|['Cuisine: Europe...|
|       18|['Cuisine: Contin...|
|       19| moist and delici...|
|       20|                NULL|
+---------+--------------------+
only showing top 20 rows



In [21]:
spark.sql("""UPDATE dishe_region SET nome='AAAAA' WHERE id_region=0""")

DataFrame[num_affected_rows: bigint]

In [22]:
result = spark.sql("SELECT * FROM dishe_region")
result.show()

+---------+--------------------+
|id_region|                nome|
+---------+--------------------+
|        0|               AAAAA|
|        2|['Cuisine: Contin...|
|        3|['Cuisine: North ...|
|        4|['Cuisine: South ...|
|        5| ['Cuisine: Indian']|
|        6| ['Cuisine: French']|
|        7| ['Cuisine: Fusion']|
|        8|['Cuisine: Bengal...|
|        9| and garlic paste...|
|       10| ['Cuisine: Indian']|
|       11|['Cuisine: Kerala...|
|       12|['Cuisine: South ...|
|       13| ['Cuisine: Indian']|
|       14|['Cuisine: South ...|
|       15| ['Cuisine: Fusion']|
|       16|['Cuisine: Goan R...|
|       17|['Cuisine: North ...|
|       18|['Cuisine: Japane...|
|       19|['Cuisine: North ...|
|       20| ['Cuisine: Indian']|
+---------+--------------------+
only showing top 20 rows

